In [1]:
import sys
import numba
import swifter
import timeit
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

__author__ = 'HK Dambanemuya'
__version__ = 'Python 2'

In [2]:
data = pd.read_csv('../Data/bid_fe.csv')
data.sample(5)

,ListingKey,BidCreationDate,MemberKey,Bid_Key,BidAmount,BidMinimumRate,BidMinimumYield,ListingStatus,Status,Member_Key,MemberCreationDate,MemberRoles,ListingCreationDate,ListingStartDate,Listing_Key,FundedOrNot,RepaidOrNot
4132178,F58E34167884889911BE4B6,2008-03-29 15:14:51,93D233810902100419DB0A9,AAE63520033296404476B70,143.56,NaN,NaN,Completed,Winning,93D233810902100419DB0A9,2007-02-10 15:11:00,Lender,2008-03-27 12:37:03,2008-03-27 12:39:18,F58E34167884889911BE4B6,True,True
1171444,AD753382971551896F4FA44,2007-03-21 06:56:24,42023382006897977EB17FA,D8383383704333705A85AF5,50.00,NaN,NaN,Completed,Winning,42023382006897977EB17FA,2007-02-27 06:26:00,Lender,2007-03-12 19:21:56,2007-03-12 19:21:56,AD753382971551896F4FA44,True,True
2469136,83BF33990435644139AC494,2007-09-10 20:11:11,18683374376001700DB8092,853435037213505320CED2B,100.00,NaN,NaN,Completed,Winning,18683374376001700DB8092,2006-11-27 12:00:00,Lender,2007-09-10 16:55:54,2007-09-10 20:06:50,83BF33990435644139AC494,True,True
1447563,94C33387132728806F5F07D,2007-04-19 02:39:43,9E753366473054690DC0E26,B1B4348987102847914368E,0.00,0.08,0.075,Completed,Outbid,9E753366473054690DC0E26,2006-08-31 18:42:00,Lender,2007-04-15 07:26:12,2007-04-15 07:35:23,94C33387132728806F5F07D,True,True
3242571,8FCE34079714922092DDE3D,2007-12-10 13:59:23,DDC73405462076208DA1B51,9E773511318817355564C6B,0.00,0.12,0.120,Completed,Outbid,DDC73405462076208DA1B51,2007-11-18 14:52:03,Lender,2007-12-10 13:52:21,2007-12-10 13:55:12,8FCE34079714922092DDE3D,True,True


In [3]:
len(data)

9733452

In [4]:
hashmap = dict()
memberkeys = sorted(list(set(data.MemberKey)))[50000:60000]
len(memberkeys)

10000

In [5]:
start = timeit.default_timer()
for key in memberkeys:
    hashmap[key] = data.query("MemberKey == '{0}'".format(key))
stop = timeit.default_timer()
print "Hashmap generated in {0} seconds".format((stop-start)) 

Hashmap generated in 3166.8285449 seconds


In [6]:
data = data[data['MemberKey'].isin(memberkeys)]

In [7]:
len(data)

1455755

## Lender Completed Listings

In [8]:
def lender_completed_listings(x):
    return hashmap[x["MemberKey"]].query("ListingStatus == 'Completed' and ListingStartDate < '{0}'".format(x["ListingStartDate"]))["ListingKey"].nunique()

start = timeit.default_timer()    
data["LenderCompletedListings"] = data[["MemberKey", "ListingStartDate"]].swifter.apply(lender_completed_listings, axis=1)
stop = timeit.default_timer()
print "Feature engineering completed in {0} seconds".format((stop-start)) 

Feature engineering completed in 4413.8087196 seconds


## Lender Repaid Listings

In [9]:
def lender_repaid_listings(x):
    return hashmap[x["MemberKey"]].query("RepaidOrNot == True and ListingStartDate < '{0}'".format(x["ListingStartDate"]))["ListingKey"].nunique()

start = timeit.default_timer()   
data["LenderRepaidListings"] = data[["MemberKey", "ListingStartDate"]].swifter.apply(lender_repaid_listings, axis=1)
stop = timeit.default_timer()
print "Feature engineering completed in {0} seconds".format((stop-start)) 

Feature engineering completed in 4162.0689696 seconds


## Lender Total Listings

In [10]:
def lender_total_listings(x):
    return hashmap[x["MemberKey"]].query("ListingStartDate < '{0}'".format(x["ListingStartDate"]))["ListingKey"].nunique()

start = timeit.default_timer()    
data["LenderTotalListings"] = data[["MemberKey", "ListingStartDate"]].swifter.apply(lender_total_listings, axis=1)
stop = timeit.default_timer()
print "Feature engineering completed in {0} seconds".format((stop-start)) 

Feature engineering completed in 4069.0011142 seconds


In [11]:
data[["MemberKey", "ListingKey", "ListingStartDate", "ListingStatus", "RepaidOrNot", "LenderCompletedListings", "LenderRepaidListings", "LenderTotalListings"]].query("MemberKey=='{0}'".format(memberkeys[0])).sort_values("ListingStartDate")

,MemberKey,ListingKey,ListingStartDate,ListingStatus,RepaidOrNot,LenderCompletedListings,LenderRepaidListings,LenderTotalListings
4210878,C2C03412691685961E8CA7B,072B3418072348034741B0D,2008-04-05 21:02:27,Completed,True,0,0,0
4211310,C2C03412691685961E8CA7B,072B3418072348034741B0D,2008-04-05 21:02:27,Completed,True,0,0,0
4212720,C2C03412691685961E8CA7B,1E7E3417368884461AEC697,2008-04-06 05:34:43,Completed,False,1,1,1
4215836,C2C03412691685961E8CA7B,BCD234170184576481196DD,2008-04-06 07:04:31,Completed,True,2,1,2
4209778,C2C03412691685961E8CA7B,529D3416499886641AA3DDD,2008-04-06 17:47:45,Completed,False,3,2,3
4217614,C2C03412691685961E8CA7B,61EF34177697686175BF35D,2008-04-06 18:00:45,Completed,False,4,2,4
4217767,C2C03412691685961E8CA7B,61EF34177697686175BF35D,2008-04-06 18:00:45,Completed,False,4,2,4
4218772,C2C03412691685961E8CA7B,1DF23417770895619297ED3,2008-04-07 07:20:16,Completed,True,5,2,5
4252159,C2C03412691685961E8CA7B,0E383416904504215A80402,2008-04-10 07:25:58,Completed,True,6,3,6
4262850,C2C03412691685961E8CA7B,C1BF3417616808413960BEE,2008-04-11 10:30:07,Cancelled,NaN,7,4,7


## Save Data

In [12]:
data.to_csv("../Data/lender_listing_fe/lender_listing_fe_60k.csv", index=False)